<a href="https://colab.research.google.com/github/mongvuot336699/DevC/blob/master/Copy_of_Week5_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [30]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [31]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv('labeledTrainData.tsv', sep='\t', encoding="latin-1")
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [33]:
df['review'].value_counts()
# loai bo <br /> , \ , - 
# lower 
# dau cham "." , dau phay "," 

How has this piece of crap stayed on TV this long? It's terrible. It makes me want to shoot someone. It's so fake that it is actually worse than a 1940s sci-fi movie. I'd rather have a stroke than watch this nonsense. I remember watching it when it first came out. I thought, hey this could be interesting, then I found out how absolutely, insanely, ridiculously stupid it really was. It was so bad that I actually took out my pocket knife and stuck my hand to the table.<br /><br />Please people, stop watching this and all other reality shows, they're the trash that is jamming the networks and canceling quality programming that requires some thought to create.                                                                                                                                                                                                                                                                                                                                                

## 2. Preprocessing

In [34]:
# stop words
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) 
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Your code here
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    # Your code here
    
    return text

In [38]:
df['review'] = df['review'].apply(preprocessor)

In [39]:
df['review']

0        with all this stuff going down at the moment w...
1         the classic war of the worlds by timothy hine...
2        the film starts with a manager nicholas bell g...
3        it must be assumed that those who praised this...
4        superbly trashy and wondrously unpretentious 8...
                               ...                        
24995    it seems like more consideration has gone into...
24996    i don t believe they made this film completely...
24997    guy is a loser can t get girls needs to build ...
24998    this 30 minute documentary buã uel made in the...
24999    i saw this movie as a child and it broke my he...
Name: review, Length: 25000, dtype: object

In [40]:
#stop words
from collections import Counter 
from nltk.corpus import stopwords 

stop_words = stopwords.words('english')
vocab = Counter()

for review in df.review:
  for word in review.split(' '):
    vocab[word] += 1

vocab.most_common(20)

[('the', 336626),
 ('and', 164097),
 ('a', 163112),
 ('of', 145848),
 ('to', 135701),
 ('is', 107310),
 ('it', 96377),
 ('in', 93953),
 ('i', 87576),
 ('this', 75964),
 ('that', 73252),
 ('s', 63915),
 ('', 49411),
 ('was', 48195),
 ('as', 46932),
 ('for', 44329),
 ('with', 44118),
 ('movie', 44020),
 ('but', 42612),
 ('film', 40135)]

In [41]:
vocab_reduced = Counter()
for w, c in vocab.items():
  if w not in stop_words:
    vocab_reduced[w] = c 

vocab_reduced.most_common(20)

[('', 49411),
 ('movie', 44020),
 ('film', 40135),
 ('one', 26783),
 ('like', 20273),
 ('good', 15140),
 ('time', 12718),
 ('even', 12645),
 ('would', 12436),
 ('story', 11976),
 ('really', 11734),
 ('see', 11474),
 ('well', 10651),
 ('much', 9762),
 ('get', 9308),
 ('bad', 9301),
 ('people', 9283),
 ('also', 9156),
 ('first', 9061),
 ('great', 9057)]

In [42]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [43]:
# TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer (
                    stop_words = stop_words,
                    tokenizer = tokenizer_porter,
                    preprocessor = preprocessor
)
X = df['review']
X_vector = tfidf.fit_transform(X)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [44]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
#X = df['review']
Xv = X_vector
y = df['sentiment']
X_train , X_test , y_train, y_test = train_test_split(Xv,y, test_size = 0.2 , random_state = 102)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [47]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer (
#                     stop_words = stop_words,
#                     tokenizer = tokenizer_porter,
#                     preprocessor = preprocessor
# )
clf = LogisticRegression(random_state=0)

clf.fit(X_train, y_train)
# clf = Pipeline(# Your code here)
# clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## 4. Evaluate Model

In [48]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix , classification_report
predict = clf.predict(X_test)
print('accuracy_score: ' , accuracy_score(y_test,predict))
print('confusion_matrix: \n', confusion_matrix(y_test, predict))
print('classification_report: \n', classification_report(y_test,predict))


accuracy_score:  0.885
confusion_matrix: 
 [[2162  341]
 [ 234 2263]]
classification_report: 
               precision    recall  f1-score   support

           0       0.90      0.86      0.88      2503
           1       0.87      0.91      0.89      2497

    accuracy                           0.89      5000
   macro avg       0.89      0.89      0.88      5000
weighted avg       0.89      0.89      0.88      5000



## 5. Export Model 

In [51]:
# Using pickle to export our trained model
import pickle
import os
pickle.dump(clf, open('logisticRegression.pkl', 'wb'))